In [44]:
from bs4 import BeautifulSoup as bsoup, Tag
import requests
import datetime
import time
import pandas as pd
import numpy as np

In [45]:
indeed_mle_csv = "indeed/indeed_mle.csv"
df = pd.read_csv(indeed_mle_csv)
df.head()

,Position,Title,Description,Url


In [46]:
def saveWebpage(prefix,response):
    """Save webpage locally as HTML and returns file location as string"""
    now = datetime.datetime.now().strftime("%Y-%m-%d_%H.%M")
    doc_name = "indeed/{prefix}_{time}.html".format(prefix=prefix,time=now)
    #https://stackoverflow.com/questions/31126596/saving-response-from-requests-to-file
    with open(doc_name, mode="wb+") as localfile:
        localfile.write(response.content)
    return doc_name

def getUrlsFromPage(website,params,prefix):
    response = requests.request("GET", website, params=params)
    page_location = saveWebpage(prefix,response)
    soup = bsoup(open(page_location), "html.parser")
    raw_urls = soup.find_all("a",class_="turnstileLink")
    print(len(soup.find("div",class_="pagination").findChildren("a",recursive=False)))
    more_urls = True if soup.find("div",class_="pagination").findChildren("a",recursive=False,limit=1) else False
    return raw_urls, more_urls

def makeUrlsVisitable(urls):
    visitable_urls = []
    #other endpoints I found are /pagead and /cmp
    acceptable_endpoints = ["rc"]
    #https://stackoverflow.com/questions/36300158/python-split-text-after-the-second-occurrence-of-character
    for url in urls:
        if (type(url) is Tag and (url["href"].split("/", 2))[1] in acceptable_endpoints):
            visitable_urls.append("https://www.indeed.com"+url["href"])
    return visitable_urls
    

def getAllUrls(website,params,prefix):
    page_num = 0
    all_urls = set()
    page_urls, more_urls = getUrlsFromPage(website,params,prefix)
    if page_urls:
        print("page {page_num}: {urls_on_page} urls on page".format(page_num=1,urls_on_page=len(page_urls)))
        u = makeUrlsVisitable(page_urls)
        for url in u:
            all_urls.add(url)
        while more_urls and page_num < 5: #self rate limiting
            params["start"] = str(int(params["start"])+50) #get next 50 results
            page_urls, more_urls = getUrlsFromPage(website,params,prefix)
            page_num += 1
#             print("page {page_num}: {urls_on_page} urls on page".format(page_num=page_num,urls_on_page=len(page_urls)))
            u = makeUrlsVisitable(page_urls)
            for url in u:
                all_urls.add(url)
            time.sleep(2)
    return all_urls
    
    
def getJobsData(urls):
    """Gets job data from each url and returns Numpy matrix of data"""
    data = np.array([])
    return data
    
def saveJobsData(data):
    """Saves jobs data to Pandas dataframe and from there to CSV file"""
    return None


In [47]:
#get MLE data
website = "https://www.indeed.com/jobs"
# mle_params = {"q":"machine+learning+engineer","l":"San+Francisco+Bay+Area,+CA","explvl":"entry_level"}
mle_params = {"as_and":"machine learning engineer","as_phr":"","as_any":"","as_not":"","as_ttl":"","as_cmp":"","jt":"all","st":"","as_src":"","salary":"","radius":"25","l":"San Francisco Bay Area, CA","fromage":"any","limit":"50","sort":"","psf":"advsrch","start":"0"}
mle_prefix = "mle" 

20
page 1: 60 urls on page
21
21
21
21
21
251


In [ ]:
urls = getAllUrls(website,mle_params,mle_prefix)
#https://stackoverflow.com/questions/8466014/how-to-convert-a-python-set-to-a-numpy-array
urls = list(urls)
num_urls = len(urls)
print(num_urls)
for url in urls:
    df.loc[len(df)] = [None,None,None,url.strip()]

In [49]:
df.to_csv(indeed_mle_csv, encoding='utf-8', index=False)

,Position,Title,Description,Url
0,None,None,None,https://www.indeed.com/rc/clk?jk=a4b96f68e3358...
1,None,None,None,https://www.indeed.com/rc/clk?jk=874c931eda801...
2,None,None,None,https://www.indeed.com/rc/clk?jk=6db9ecef73f21...
3,None,None,None,https://www.indeed.com/rc/clk?jk=cbeae73193a4c...
4,None,None,None,https://www.indeed.com/rc/clk?jk=99b1587ae9242...
5,None,None,None,https://www.indeed.com/rc/clk?jk=68d2210334775...
6,None,None,None,https://www.indeed.com/rc/clk?jk=85b912f3f9cd7...
7,None,None,None,https://www.indeed.com/rc/clk?jk=4dec418aecacc...
8,None,None,None,https://www.indeed.com/rc/clk?jk=9a2eca4654e14...
9,None,None,None,https://www.indeed.com/rc/clk?jk=4094194e91b8e...


In [55]:
df.iloc[0]["Url"]

'https://www.indeed.com/rc/clk?jk=a4b96f68e3358b32&fccid=f89deb5a97c7738a&vjs=3'

In [ ]:
# headers = {'User-Agent': 'Mozilla/5.0'}
# # mle_params = {"as_and":"machine learning engineer","as_phr":"","as_any":"","as_not":"","as_ttl":"","as_cmp":"","jt":"all","st":"","as_src":"","salary":"","radius":"25","l":"San Francisco Bay Area, CA","fromage":"any","limit":"50","sort":"","psf":"advsrch","start":"0"}
# response = requests.get("https://www.indeed.com/jobs?q=machine+learning+engineer&explvl=entry_level", headers = headers)

# soup = BeautifulSoup(response.text,'lxml')

# #This will you all a tags in div that has pagination class
# pages = soup.find("div",class_='pagination').findChildren("a",recursive=False,limit=1)
# "res",pages
# import requests

# url = "https://www.indeed.com/jobs"

# querystring = {"as_and":"machine+learning+engineer","as_phr":"","as_any":"","as_not":"","as_ttl":"","as_cmp":"","jt":"all","st":"","as_src":"","salary":"","radius":"25","l":"San+Francisco+Bay+Area%2C+CA","fromage":"any","limit":"50","sort":"","psf":"advsrch"}

# headers = {
#     'cache-control': "no-cache",
#     'Postman-Token': "3e4c7fb2-cfc6-4b86-8351-e605858228b1"
#     }

# response = requests.request("GET", url, headers=headers, params=querystring)
# soup = BeautifulSoup(response.text,'lxml')

In [68]:
test = bsoup(requests.get(df.iloc[0]["Url"]).text, "html.parser")
print(test.find("h3",class_="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title").string)

ACTV Machine Learning Engineer Intern
